In [1]:
from mlrun import new_function, code_to_function, NewTask, get_run_db, mlconf, mount_v3io, new_model_server, builder
import kfp
from kfp import dsl

In [2]:
mlconf.dbpath = 'http://mlrun-api:8080'

## Run Local
Run one of the functions in this notebook with mlrun. This execution happens in the notebook container. As per the dbpath config execution is stored in the rundb. 

In [ ]:
#task = NewTask('run_local', handler="nb1f1",out_path='/User/mlrun/data')
#run = code_to_function('functions_file',kind='local',filename="function1.py").run(task)

workflow = code_to_function('functions_file',kind='local',filename="/User/mlrun_samples/function1.py")

In [ ]:
print (workflow.to_yaml())

In [ ]:
workflow.spec.command="/User/mlrun_samples/function1.py"
function1 = workflow.run(
    name='function1',
    handler='nb1f1').apply(mount_v3io())

In [ ]:
print(run.to_yaml())

## Run in container
First you need to Build an image for your functions to run on.

In [4]:
docker_registry = "docker-registry.default-tenant.app.mdl0112a.iguazio-cd2.com:80"

In [5]:
workflow = code_to_function('functions_file',kind='job',filename="/User/mlrun_samples/function1.py").apply(mount_v3io())

## Build image once

In [6]:
builder.build_image(docker_registry + '/ecolab/img:latest',commands=['pip install v3io_frames'])

FROM python:3.6-jessie
RUN pip install v3io_frames
RUN pip install mlrun

[mlrun] 2020-02-17 20:41:21,559 using in-cluster config.
[mlrun] 2020-02-17 20:41:21,584 Pod mlrun-build-lm8rm created
..
INFO[0000] Resolved base name python:3.6-jessie to python:3.6-jessie 
INFO[0000] Resolved base name python:3.6-jessie to python:3.6-jessie 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:0318d80cb241983eda20b905d77fa0bfb06e29e5aabf075c7941ea687f1c125a: no such file or directory 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:0318d80cb241983eda20b905d77fa0bfb06e29e5aabf075c7941ea687f1c125a: no such file or directory 
INFO[0000] Downloading base image python:3.6-jessie     
INFO[0001] Un

'succeeded'

In [7]:
workflow.spec.image= docker_registry + '/ecolab/img:latest'

### Turn notebook into function code
Now you can run the same function on a container outside of the notebook container

In [8]:
# create an ML function from the notebook, attache it to iguazio data fabric (v3io)
workflow.spec.command="/User/mlrun_samples/function1.py"
function1 = workflow.run(
    name='function1',
    handler='nb1f1')

[mlrun] 2020-02-17 20:43:39,224 starting run function1 uid=e678bbecb1c84c76a6301bc312b7022d  -> http://mlrun-api:8080
[mlrun] 2020-02-17 20:43:39,377 Job is running in the background, pod: function1-gxmv7
None None

[mlrun] 2020-02-17 20:43:55,710 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...b7022d,0,Feb 17 20:43:55,completed,function1,host=function1-gxmv7kind=jobowner=admin,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run e678bbecb1c84c76a6301bc312b7022d  , !mlrun logs e678bbecb1c84c76a6301bc312b7022d 
[mlrun] 2020-02-17 20:43:58,710 run executed, status=completed


In [ ]:
# View generated fucntion
print(workflow.to_yaml())

## Execute multiple functions in a pipeline
You can build a pipeline with the individual functions in a notebook as each step. 

In this case:
1. Execute nb1f1
2. Wait for nb1f1
3. Execute nb1f1 and nb2f1.
4. Wait for completion of both
5. Execute write_to_kv with hyper parameters.


## Merge all scripts to provide a single function code source

In [9]:
!cat common.py function1.py function2.py write_kv.py > /tmp/allfunctions.py

In [10]:
# Initialize the fucntion once for all functions in a notebook
fn = code_to_function('functions_file',kind='job',filename="/tmp/allfunctions.py").apply(mount_v3io())
fn.spec.image = docker_registry + '/ecolab/img:latest'



@dsl.pipeline(
    name='In notebook functions pipeline definition',
    description='Shows how to use mlrun.'
)
def example_pipeline(
   p1 = [1,2,3,4,5,6],
   p2 = [9,8,6,5,4,3]
):
    # Use the same fn definition, but run different functions.
    # fn2 is a definition from the external notebook
    f1 = fn.as_step(NewTask(handler='nb1f1', params={'p1': p1,'p2': p2 }), name='nb1f1').apply(mount_v3io())
    f2 = fn.as_step(NewTask(handler='nb1f2', params={'p1': p1}),name ='nb1f2').apply(mount_v3io()).after(f1)
    f3 = fn.as_step(NewTask(handler='write_to_kv').with_hyper_params({'p1':p1,'p2':p2}),name="with_hyper_param").\
                    apply(mount_v3io()).after(f2).after(f2)
    

In [14]:
client = kfp.Client(namespace='default-tenant')
p1 = [1,2,3,4,5,6]
p2 = [9,8,6,5,4,3]
#arguments = {'p1': p1 ,'p2': p2}
arguments={'jake':'isfake'}

In [ ]:
run_result = client.create_run_from_pipeline_func(example_pipeline, arguments, run_name='example 1', experiment_name='example')

In [ ]:
!curl -X GET http://${DOCKER_REGISTRY_SERVICE_HOST}:${DOCKER_REGISTRY_SERVICE_PORT}/v2/_catalog

In [ ]:
!curl -X GET http://docker-registry.default-tenant.app.mdl1028.iguazio-cd2.com/v2/_catalog